https://developer.company-information.service.gov.uk/

# OAuth 2.0 with Companies House API

This notebook will guide you through setting up OAuth 2.0 authentication for the Companies House API, which includes registering an application, obtaining access tokens, and making authenticated API requests.

## Step-by-Step Instructions:

1. **Register Your Application**: Create an application in the Companies House Developer Portal to get `client_id` and `client_secret`.
2. **Generate Authorization URL**: Create a URL to redirect users to for authorization.
3. **Set Up Web Server**: Handle redirects and capture the authorization code.
4. **Exchange Code for Token**: Use the authorization code to get access and refresh tokens.
5. **Make API Requests**: Use the access token to make authenticated requests to the API.

Let's start with the setup and registration.


# OAuth 2.0 with Companies House API for Dairy and Cattle Farm Data

## Step 1: Register Your Application
- Go to the Companies House Developer Portal.
- Sign up or log in and then register a new application (I called it "CDH Farm Data UK Project").
   - the application can be either set up as Test or Live. The difference is that the Test application will only have access to the test environment, while the Live application will have access to the live environment. In practice, this means that the Test application will only be able to access the test company data, while the Live application will be able to access the live company data. Here we chose 'Live'.
   -  We then create a key, I am calling mine "Giulia1". We are asked to select the type of API client key we want to create, "REST" "Streaming" or "Web". This is a REST API, so we chose "REST".  We are then asked to provide "Restricted IPs" and "JavaScript domains". We leave these blank.
      - once created we can access the API key. In my case this is  "669fcc9f-7d0f-4566-a9c5-002f4a3f2aee"

- Note down your client_id, client_secret, and specify your redirect_uri (e.g., http://localhost:5000/callback).

## Step 2: Set Up and Authenticate with OAuth 2.0
- Constants and Import Statements

First, set up the required constants and import the necessary libraries.

First, the developer needs to create a web client for their application obtaining a "client_id" and a "client_secret" which must be stored securely by the developer.

The web server application must be configured to use this client_id and client_secret combination for interactions with the Companies House OAuth 2.0 service.

We can  use the "Authorization Code" grant type to obtain an access token from the Companies House OAuth 2.0 service. This involves the following steps:

1. The web server application redirects the user to the Companies House OAuth 2.0 service for authorization.
2. The user logs in and authorizes the web server application.
3. The Companies House OAuth 2.0 service redirects the user back to the web server application with an authorization code.
4. The web server application exchanges the authorization code for an access token.
5. The web server application can now use the access token to make authenticated requests to the Companies House API.
6. The access token can be used to make requests to the Companies House API on behalf of the user.
7. The access token can be refreshed using the refresh token when it expires.
8. The refresh token can be used to obtain a new access token without requiring the user to log in again.
9. The access token and refresh token must be stored securely by the web server application.

In practice, you need to set up a web server to handle the OAuth 2.0 redirect and capture the authorization code. This can be done using a web framework like Flask. The web server will need to handle the following endpoints: 
- `/`: The home page that redirects the user to the Companies House OAuth 2.0 service for authorization.
- `/callback`: The endpoint that receives the authorization code from the Companies House OAuth 2.0 service.
- `/exchange_token`: The endpoint that exchanges the authorization code for an access token.
- The web server will need to store the access token and refresh token securely for future use.
- The web server will need to handle rate limiting to comply with the Companies House API rate limits.
- The web server will need to implement functions to fetch data from the Companies House API using the access token.
- The web server will need to implement functions to prepare the data for social network analysis.
- The data can be saved to a CSV file for further analysis or teaching purposes.

The Companies House API provides access to a wealth of company data, including information on officers, filings, and financials. By using OAuth 2.0 to authenticate with the API, you can access this data securely and make authenticated requests on behalf of users.

In this notebook, we will walk through the process of setting up OAuth 2.0 authentication with the Companies House API, fetching data on dairy and cattle farms, and preparing the data for social network analysis.

Let's get started!

In [ ]:
# Install the necessary package if not already installed
# !pip install flask requests

In [ ]:
import requests
import time
from flask import Flask, request, redirect, url_for
import urllib.parse
import pandas as pd

# Constants (replace with your actual values)
CLIENT_ID = 'YOUR_CLIENT_ID'  # Replace with your client ID. You can find this in the developer portal.
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'  # Replace with your client secret key. You can find this in the developer portal.
REDIRECT_URI = 'http://localhost:5000/callback'  # Adjust this as needed
AUTH_URL = 'https://account.companieshouse.gov.uk/oauth2/authorise'
TOKEN_URL = 'https://account.companieshouse.gov.uk/oauth2/token'
BASE_API_URL = 'https://api.company-information.service.gov.uk'
SCOPE = 'read-public-data'


# Step 3: Generate Authorization URL
Construct the URL that users will visit to authorize your application.

In [ ]:
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'redirect_uri': REDIRECT_URI,
    'scope': SCOPE
}

authorization_url = f"{AUTH_URL}?{urllib.parse.urlencode(params)}"
print(f"Visit this URL to authorize: {authorization_url}")

# Step 4: Set Up a Web Server to Handle Redirects

Use Flask to handle the OAuth redirect and capture the authorization code.

In [ ]:
app = Flask(__name__)

@app.route('/')
def home():
    return f'<a href="{authorization_url}">Click here to authorize</a>'

@app.route('/callback')
def callback():
    code = request.args.get('code')
    if code:
        return redirect(url_for('exchange_token', code=code))
    return 'Authorization failed.'

@app.route('/exchange_token')
def exchange_token():
    code = request.args.get('code')
    tokens = exchange_code_for_token(code)
    access_token = tokens['access_token']
    refresh_token = tokens['refresh_token']
    return f'Access Token: {access_token}<br>Refresh Token: {refresh_token}'

def exchange_code_for_token(code):
    data = {
        'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': REDIRECT_URI,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET
    }
    response = requests.post(TOKEN_URL, data=data)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# Uncomment the next line to run the Flask app
# app.run(debug=True, port=5000)

Run the Flask server, navigate to http://localhost:5000, and complete the OAuth flow to get your access token.

Step 5: Exchange Authorization Code for Access Token
After obtaining the authorization code, exchange it for an access token.

In [ ]:
# Example usage (replace 'AUTHORIZATION_CODE' with the actual code)
authorization_code = 'AUTHORIZATION_CODE'
tokens = exchange_code_for_token(authorization_code)
access_token = tokens.get('access_token')
refresh_token = tokens.get('refresh_token')
print(f"Access Token: {access_token}")
print(f"Refresh Token: {refresh_token}")

# Step 6: Fetch Data for Dairy and Cattle Farms
Create a function to retrieve company details using the access token.

In [ ]:
def fetch_company_details(access_token, company_number):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    url = f"{BASE_API_URL}/company/{company_number}"
    response = requests.get(url, headers=headers)
    return response.json()


Retrieve Companies by SIC Code

Use the SIC codes for dairy and cattle farms to search for relevant companies and incorporate rate limiting.

In [ ]:
def search_companies_by_sic(access_token, sic_codes):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    companies_data = []
    request_count = 0
    start_time = time.time()

    for sic_code in sic_codes:
        url = f"{BASE_API_URL}/search/companies?q={sic_code}&items_per_page=100"
        response = requests.get(url, headers=headers)
        request_count += 1
        if response.status_code == 200:
            data = response.json()
            companies_data.extend(data.get('items', []))
        
        # Handle rate limiting
        if request_count >= 600:
            elapsed_time = time.time() - start_time
            if elapsed_time < 300:  # 5 minutes
                time.sleep(300 - elapsed_time)
            request_count = 0
            start_time = time.time()
    
    return companies_data

# SIC Codes for dairy and cattle farms
sic_codes = ['01410', '01420']
companies = search_companies_by_sic(access_token, sic_codes)
print(companies[:5])  # Print the first 5 companies

# Step 7: Retrieve and Analyze Officer Data

Create a function to fetch officer details for each company.

In [ ]:
def fetch_company_officers(access_token, company_number):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    url = f"{BASE_API_URL}/company/{company_number}/officers"
    response = requests.get(url, headers=headers)
    return response.json().get('items', [])


Analyze Data for Social Network Analysis

Prepare the data for social network analysis, focusing on relationships between company officers.

In [ ]:
def prepare_sna_data(companies, access_token):
    sna_data = []
    request_count = 0
    start_time = time.time()

    for company in companies:
        company_number = company.get('company_number')
        officers = fetch_company_officers(access_token, company_number)
        request_count += 1

        for officer in officers:
            sna_data.append({
                'company_name': company.get('title'),
                'company_number': company_number,
                'officer_name': officer.get('name'),
                'officer_role': officer.get('officer_role'),
                'appointed_on': officer.get('appointed_on'),
                'resigned_on': officer.get('resigned_on')
            })
        
        # Handle rate limiting
        if request_count >= 600:
            elapsed_time = time.time() - start_time
            if elapsed_time < 300:  # 5 minutes
                time.sleep(300 - elapsed_time)
            request_count = 0
            start_time = time.time()
    
    return pd.DataFrame(sna_data)

sna_df = prepare_sna_data(companies, access_token)
sna_df.head()

#  Step 8: Save and Document Data

Save the collected and prepared data to a CSV file for further analysis or teaching purposes.

In [ ]:
sna_df.to_csv('sna_data.csv', index=False)
print("Data saved to 'sna_data.csv'")

## Documentation for OAuth 2.0 with Companies House API

This notebook demonstrates how to authenticate with the Companies House API using OAuth 2.0, fetch data on dairy and cattle farms, and prepare the data for social network analysis.

Steps:
- Register Application: Obtain client_id and client_secret.
- Generate Authorization URL: Create a URL for users to authorize the application.
- Set Up Web Server: Handle OAuth redirects and capture the authorization code.
- Exchange Code for Token: Get access token using the authorization code.
- Fetch Data: Retrieve company details and officer information.
- Prepare Data: Format the data for social network analysis.
- Save Data: Store the collected data in a CSV file.
- Rate Limiting: Pause requests to comply with the API's rate limit.
- Key Functions: Implement functions for exchanging tokens, fetching data, and preparing SNA data.

Notes:
- Replace placeholders with actual credentials.
- Ensure Flask is set up correctly for handling OAuth redirects.
- Manage tokens securely and handle expiration appropriately.
